In [ ]:
from matplotlib import pyplot as plt
import intake
import numpy as np
import xarray as xr
import pandas as pd
import cftime
import os

class input_params:
    def __init__(self, exp, table, variables, model, institution, member):
        self.exp = exp
        self.table = table
        self.variables = variables
        self.model = model
        self.institution = institution
        self.member = member
        
# institution: NCAR  model: CESM2
# institution: NOAA-GFDL  model: GFDL-CM4

#models = dataframe.search(experiment_id=exp,
          #                source_id=model,institution_id=institution)
#print(models.df.variable_id.unique())

#### CESM2-WACCM has both ssp585 and ssp245, but it did not have va





#input_obj = input_params('ssp245', 'day', 'ua', 'GFDL-CM4', 'NOAA-GFDL', 'r1i1p1f1') Done
#input_obj = input_params('ssp245', 'day', 'va', 'GFDL-CM4', 'NOAA-GFDL', 'r1i1p1f1') Done
#input_obj = input_params('ssp245', 'day', 'pr', 'GFDL-CM4', 'NOAA-GFDL', 'r1i1p1f1') Done
#input_obj = input_params('ssp245', 'day', 'psl', 'GFDL-CM4', 'NOAA-GFDL', 'r1i1p1f1') Done




#input_obj = input_params('ssp585', 'day', 'ua', 'GFDL-CM4', 'NOAA-GFDL', 'r1i1p1f1') Done
#input_obj = input_params('ssp585', 'day', 'va', 'GFDL-CM4', 'NOAA-GFDL', 'r1i1p1f1')   Done
#input_obj = input_params('ssp585', 'day', 'pr', 'GFDL-CM4', 'NOAA-GFDL', 'r1i1p1f1') Done
# input_obj = input_params('ssp585', 'day', 'psl', 'GFDL-CM4', 'NOAA-GFDL', 'r1i1p1f1') Done


url="https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
dataframe = intake.open_esm_datastore(url)


# models = dataframe.search(source_id=input_obj.model,
#                           institution_id=input_obj.institution)



# print(models.df.experiment_id.unique())
# #print("####################################")
# print(models.df.source_id.unique())
# print(models.df.variable_id.unique())
# print(models.df.member_id.unique())
# print(models.df.institution_id.unique())
# print(models.df.table_id.unique())

models = dataframe.search(experiment_id=input_obj.exp,
                          table_id=input_obj.table,
                          variable_id=input_obj.variables,
                          source_id=input_obj.model,
                          member_id=input_obj.member,
                          institution_id=input_obj.institution)

print("Here1")
datasets = models.to_dataset_dict()

exp= input_obj.exp
table= input_obj.table
variables= input_obj.variables
model= input_obj.model
institution= input_obj.institution
member= input_obj.member

# Set output path information
base_path='/shared/cmip6/data/'
out_path=base_path+exp+'/atmos/day/'+variables+'/'+institution+'.'+model+'/'+member+'/'

# Check to see if directory exists
if not os.path.exists(out_path):
    print("Making Directory: ", out_path)
    os.makedirs(out_path)

print(datasets.keys())
for keys in datasets.keys():
    ds = datasets[keys]
    time_str=ds['time'].dt.strftime("%Y%m%d").values
    #print(ds['plev'])
    # Winds (extract levels)
    print(variables)
    if (variables=='ua' or variables=='va'):

        # Extract the levels
        ds_200=ds.sel(plev=25000.)
        ds_850=ds.sel(plev=85000.)
        
        # Combine data for two levels together
        ds_lev=xr.combine_nested([ds_200,ds_850],concat_dim='plev')

        # Write out the data
        out_file=out_path+keys+'.'+time_str[0]+'-'+time_str[-1]+'.nc'
        ds.to_netcdf(out_file)
        
    # Precip
    else:
        # Just write out the data
        out_file=out_path+keys+'.'+time_str[0]+'-'+time_str[-1]+'.nc'
        ds.to_netcdf(out_file)

In [ ]:
{GFDL-ESM4' there is no ur and vr} -- {'GFDL-CM4'} -- 'GFDL-AM4' 'GFDL-CM4C192' 'GFDL-OM4p5B'

## 

In [ ]:
#CESM2-WACCM' 'CESM2' ['CESM2-FV2' no ssp] 'CESM2-WACCM-FV2' no ssp 'CESM1-1-CAM5-CMIP5